# Tutorial for FedLab users

This is a comprehensive tutorial for users who would like to know FedLab. FedLab is built on the top of [torch.distributed](torch.distributed) modules and provides the necessary modules for FL simulation, including communication, compression, model optimization, data partition, and other functional modules. FedLab users can build FL simulation environment with custom modules like playing with LEGO bricks. 

In this tutorial, we will further describe the architecture of FedLab and its usage. To put it simply, we introduce FedLab by implementing a vanilla federated learning algorithm FedAvg in this file.

In [ ]:
# configuration
from munch import Munch
args = Munch

args.total_client = 100
args.alpha = 0.5
args.seed = 42
args.preprocess = True

## Prepare your dataset

FedLab provide necessary module for uses to patition their datasets. Additionally, various implementation of datasets partition for federated learning are also availiable at the [URL](https://github.com/SMILELab-FL/FedLab/tree/master/fedlab/dataset).

In [ ]:
# We provide a example usage of patitioned MNIST dataset
# Download raw MNIST dataset and partition them according to given configuration

from torchvision import transforms
from fedlab.dataset.partitioned_mnist import PartitionedMNIST

fed_mnist = PartitionedMNIST(root="./tests/data/mnist/",
                         path="./tests/data/mnist/fedmnist/",
                         num_clients=args.total_client,
                         partition="noniid-labeldir",
                         dir_alpha=args.alpha,
                         seed=args.seed,
                         preprocess=args.preprocess,
                         download=True,
                         verbose=True,
                         transform=transforms.Compose(
                             [transforms.ToPILImage(), transforms.ToTensor()]))


dataset = fed_mnist.get_dataset(0) # get the 0-th client's dataset
dataloader = fed_mnist.get_dataloader(0, batch_size=128) # get the 0-th client's dataset loader with batch size 128

## 1. Client local training

Client training procedure is implemented by class ClientTrainer in FedLab. We have built-in FedAvg implementation in FedLab.

In [ ]:
# client
from fedlab.contrib.clients import SGDSerialTrainer, SGDClientTrainer
from fedlab.models.mlp import MLP

model = MLP(784, 10)

trainer = SGDSerialTrainer(model, args.total_client, cuda=True) # serial trainer
# trainer = SGDClientTrainer(model, cuda=True) # single trainer

trainer.setup_dataset(fed_mnist)
trainer.setup_optim(args.epochs, args.batch_size, args.lr)

## 2. Server global aggregation

In [ ]:
# server
from fedlab.contrib.servers.server import SyncServerHandler

args.com_round = 10
args.sample_ratio = 0.1

handler = SyncServerHandler(model=model, global_round=args.com_round, sample_ratio=args.sample_ratio)

## Choose simulation mode

We provide three basic simulation mode in FedLab. Depending on the needs of users.

1. Choose Standalone mode to run the simulation with lowest resourch allocation.
2. Choose Cross-process mode to run the simulation with multi-machine or multi-gpus with faster calculation.
3. Chosse Hierachical mode to run the simulation across computer clusters.

### Standalone

We provide an example pipeline implementation. Please see [URL](https://github.com/SMILELab-FL/FedLab/blob/master/fedlab/core/standalone.py)

In [ ]:
from fedlab.core.standalone import StandalonePipeline

standalone = StandalonePipeline(handler=handler, trainer=trainer)
standalone.main()

### Cross-process

### Hierachical